#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:180%; text-align:left;padding:3.0px; background: #ccf5ff; border-bottom: 8px solid #0047b3" > TABLE OF CONTENTS<br><div>  
* [IMPORTS AND INSTALLATIONS](#1)
* [INTRODUCTION](#2)
    * [UTILITIES](#2.1)
    * [DATASET DETAILS](#2.2)    
    * [CONFIGURATION](#2.3)
    * [VERSION DETAILS](#2.4)
* [PREPROCESSING](#3)
* [MODEL TRAINING](#4)         

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #b3e0ff; border-bottom: 8px solid #1a0d00" > PACKAGE IMPORTS AND INSTALLATIONS<br> <div>

In [ ]:
%%time

##################################################################
from IPython.display import clear_output;
from gc import collect;
from google.colab import drive;
print();
drive.mount('/content/drive');

from google.colab import files;
files.upload();
!ls -lha kaggle.json;
!pip install -q kaggle --upgrade;
!mkdir -p ~/.kaggle;
!cp kaggle.json ~/.kaggle/
!pwd;
!chmod 600 ~/.kaggle/kaggle.json;
print();

##################################################################
!kaggle kernels output andreynesterov/lightautoml-038-dependencies -p /HomeCredit;
!pip install -q /HomeCredit/lightautoml-0.3.8-py3-none-any.whl;

!pip install -q --upgrade polars==0.20.5;
!pip install -q joblib;
!pip install -q colorama;
!pip install -q optuna;
!pip install -q ctypes;
!pip install -q category_encoders;
clear_output();

import lightgbm as lgb, sklearn as sk, pandas as pd, polars as pl, numpy as np;
print(f"sklearn = {sk.__version__} | numpy = {np.__version__} | lightgbm = {lgb.__version__}");
print(f"polars = {pl.__version__} | pandas = {pd.__version__}\n\n");

collect();

sklearn = 1.2.2 | numpy = 1.25.2 | lightgbm = 3.2.1

polars = 0.20.5 | pandas = 1.5.3





CPU times: user 4.06 s, sys: 1.44 s, total: 5.5 s

Wall time: 4min 29s


60

In [ ]:
%%time

##################################################################
# Downloading the competition data:-

!kaggle competitions download -c home-credit-credit-risk-model-stability -p /HomeCredit
!unzip /HomeCredit/home-credit-credit-risk-model-stability.zip -d /HomeCredit;

import shutil;
try:
    shutil.rmtree(f"/HomeCredit/csv_files");
except:
    pass;

##################################################################

print();
collect();
clear_output();

CPU times: user 1.03 s, sys: 3.98 s, total: 5.01 s

Wall time: 2min 53s


In [ ]:
%%time

from gc import collect;
from warnings import filterwarnings;
filterwarnings('ignore');
from IPython.display import display_html, clear_output;
import ctypes;
libc = ctypes.CDLL("libc.so.6");

from pprint import pprint;
from functools import partial;

from copy import deepcopy;
import pandas as pd, numpy as np, polars as pl, os, joblib;
import polars.selectors as cs;

from os import path, walk, getpid;
from psutil import Process;
import re;
from collections import Counter;
from itertools import product;
from glob import glob;

from colorama import Fore, Style, init;
from warnings import filterwarnings;
filterwarnings('ignore');
from tqdm.notebook import tqdm;

print();
collect();



CPU times: user 101 ms, sys: 0 ns, total: 101 ms

Wall time: 101 ms


0

In [ ]:
%%time

# Pipeline specifics:-
from sklearn.model_selection import (StratifiedGroupKFold as SGKF, cross_val_score, cross_val_predict);
from sklearn.pipeline import Pipeline;
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin;

# ML Model training:-
from sklearn.metrics import roc_auc_score, make_scorer;
from sklearn.ensemble import VotingClassifier as VC;

import torch;
import torch.nn as nn;
from lightautoml.automl.presets.tabular_presets import TabularAutoML;
from lightautoml.tasks import Task;

clear_output();

import xgboost as xgb, lightgbm as lgb, catboost as cb, sklearn as sk;
print(f"\nXGBoost = {xgb.__version__} | LightGBM = {lgb.__version__} | Catboost = {cb.__version__}");
print(f"Pandas = {pd.__version__} | Sklearn = {sk.__version__}| Polars = {pl.__version__}\n\n");
collect();



XGBoost = 2.0.3 | LightGBM = 3.2.1 | Catboost = 1.2.2

Pandas = 1.5.3 | Sklearn = 1.2.2| Polars = 0.20.5





CPU times: user 2.9 s, sys: 452 ms, total: 3.35 s

Wall time: 7.52 s


60

In [ ]:
%%time

# Making sklearn pipeline outputs as dataframe:-
from sklearn import set_config;
set_config(transform_output = "pandas");
pd.set_option('display.max_columns', 50);
pd.set_option('display.max_rows', 50);
pd.set_option('display.precision', 3);

# Setting global configurations for polars:-
pl.Config.activate_decimals(True).set_tbl_hide_column_data_types(True);
pl.Config(**dict(tbl_formatting = 'ASCII_FULL_CONDENSED',
                 tbl_hide_column_data_types = True,
                 tbl_hide_dataframe_shape = True,
                 fmt_float = "mixed",
                 tbl_cell_alignment = 'CENTER',
                 tbl_hide_dtype_separator = True,
                 tbl_cols = 100,
                 tbl_rows = 50,
                 fmt_str_lengths = 100,
                )
         );

CPU times: user 841 µs, sys: 0 ns, total: 841 µs

Wall time: 5.25 ms


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #b3e0ff; border-bottom: 8px solid #1a0d00" > INTRODUCTION<br> <div>

In [ ]:
%%time

class Utility:
    """
    This class serves to do the below-
    1. Define method to print in color
    2. Define the classifier metric, custom scorer callable and competition metrics
    3. Define the garbage cleaning process
    """;

    def PrintColor(self,text:str, color = Fore.BLUE, style = Style.BRIGHT):
        "Prints color outputs using colorama using a text F-string";
        print(style + color + text + Style.RESET_ALL);

    def ScoreMetric(self, ytrue:np.array, ypred: np.array)-> float:
        """
        This method calculates the classifier metric to evaluate the base-model
        Inputs- ytrue, ypred:- np.array - input true and predictions arrays
        Output- float:- base classifier metric, here- GINI score
        """;
        return roc_auc_score(ytrue, ypred);

    def StabilityMetric(self, Stb_Prf: pd.DataFrame, w_fallingrate = 88, w_resstd = -0.5)-> float:
        """
        This method calculates the GINI stability metric as below-
        1. Creates an array of GINi scores from week4-91 for in-time testing
        2. Creates a regression fit-line using numpy.polyfit
        3. Calculates the stability measure using the formula mentioned in the competition pinned notebook
        """;

        y     = Stb_Prf.groupby("WEEK_NUM").apply(lambda x: 2 * self.ScoreMetric(x[CFG.target], x["score"]) -1).values;
        x     = range(len(y));
        a, b  = np.polyfit(x, y, 1);
        y_hat = a * x + b;

        return np.mean(y) + w_fallingrate * min(0, a) + w_resstd * np.std(y - y_hat);

    def CleanMemory(self):
        "This method cleans the memory off unused objects and displays the cleaned state RAM usage";

        collect();
        libc.malloc_trim(0);
        pid        = getpid();
        py         = Process(pid);
        memory_use = py.memory_info()[0] / 2. ** 30;
        return f"\nRAM usage = {memory_use :.4} GB";

    def PredictBatch(self, model, X: pd.DataFrame, prd_proba_req: bool = True, batch_size: int = 1000)-> np.array:
        """
        This method predicts from the model in batches instead of the complete test set to avoid OOM issues in the test set
        Inputs:-
        1. X:- Train/ Test set
        2. prd_proba_req:- need predict proba/ predictions- True [predict_proba], False[predict]
        3. batch_size:- batch size to consider in one attempt

        Returns:-
        preds:- array of predicted probabilities
        """;

        num_samples = len(X);
        num_batches = int(np.ceil(num_samples / batch_size));
        preds       = np.zeros((num_samples,));

        for batch_idx in range(num_batches):
            self.PrintColor(f"---> Processing batch: {batch_idx+1}/{num_batches}", color = Fore.CYAN);

            start_idx = batch_idx * batch_size;
            end_idx   = min((batch_idx + 1) * batch_size, num_samples);
            X_batch   = X.iloc[start_idx : end_idx];

            if prd_proba_req == True:
                batch_probs = model.predict_proba(X_batch)[:, 1];
            else:
                batch_probs = model.predict(X_batch).data.squeeze();

            preds[start_idx: end_idx] = batch_probs;
            _ = self.CleanMemory();

        return preds;

Utils = Utility();
print();




CPU times: user 92 µs, sys: 0 ns, total: 92 µs

Wall time: 97.5 µs


**Data columns**<br>
This is available in the original data description as below<br>
https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/data <br>
<br>**Competition details and notebook objectives**<br>
1. This is a binary classification challenge to predict home loan credit defaulters. **GINI** is the metric for the base classifier in this challenge<br>
2. We also have to additionally assess the stability of GINI measure across time in the evaluation period. We need to score the classifier on a weekly basis and then assess the stability of the weekly GINI score using a regression model against time. Stability measure penalizes models that wane off in prediction capabilities across time. <br>
2. In this starter notebook, we start the assignment with a simple preprocessing, understanding the data structure of the competition data, basic feature emgineering and develop starter models to initiate the challenge. We will also incorporate other opinions and approaches as we move along the challenge.<br>
<br>
**Model strategy** <br>
We start off with simple tree based ML models and Denselight model-LAMA and a soft-voting ensemble with appropriate inference in the test set submission. <br>
<br>**References** <br>
1. https://www.kaggle.com/code/darynarr/home-credit-drop-date-features <br>
2. https://www.kaggle.com/code/jetakow/home-credit-2024-starter-notebook <br>
3. https://www.kaggle.com/code/jirkaborovec/credit-risk-lgbm-optuna-hyper-params <br>
4. https://www.kaggle.com/code/jirkaborovec/credit-risk-eda-xgboost-depth-0-1-gpu <br>
5. https://www.kaggle.com/code/peizhengwang/lgb-xgb-cat-ensemble-baseline <br>
6. https://www.kaggle.com/code/andreynesterov/home-credit-baseline-training-lightautoml <br>

<a id="2.3"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background:  #008080; border-bottom: 8px solid #001a1a" > CONFIGURATION<br><div>

In [ ]:
%%time

# Configuration class:-
class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    """;

    exp_nb             = 2;
    version_nb         = 6;
    test_req           = "N";
    test_sample_frac   = 0.025;
    state              = 42;
    target             = 'target';
    train_path         = "/HomeCredit/parquet_files/train";
    test_path          = "/HomeCredit/parquet_files/test";
    path               = "/kaggle/input/home-credit-credit-risk-model-stability";
    model_path         = f"/content/drive/MyDrive/HomeCreditQuality";
    null_cutoff        = 0.85;
    cat_cutoff         = 200;
    n_splits           = 3 if test_req == "Y" else 5;
    n_repeats          = 1 ;
    nbrnd_erly_stp     = 100;
    blend_wgt          = [0.15, 0.15, 0.30, 0.20, 0.20];

    # Global variables for plotting:-
    grid_specs = {'visible': True, 'which': 'both', 'linestyle': '--',
                  'color': 'lightgrey', 'linewidth': 0.75
                  };
    title_specs = {'fontsize': 9, 'fontweight': 'bold', 'color': 'tab:blue'};

print();
Utils.PrintColor(f"--> Configuration done!");
_ = Utils.PrintColor(Utils.CleanMemory());



--> Configuration done!



RAM usage = 0.8099 GB

CPU times: user 219 ms, sys: 0 ns, total: 219 ms

Wall time: 218 ms


<a id="2.4"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background:  #008080; border-bottom: 8px solid #001a1a" > VERSION DETAILS<br><div>

|Experiment <br> Number|Version|Details|Features| Models|CV score| Stability Score|Public LB score|
|:-:|:-:|---|:-:|:-:|:-:|:-:|:-:|
|2|1|* Used [public notebook](https://www.kaggle.com/code/batprem/home-credit-risk-mode-utility-scripts/notebook) features <br> * Designed LGBM XGB on feature subsets and batch predictions on multiple feature subsets <br> * Null cutoff = 50%|325|LGBM x 2 <br> XGB x 1|0.82743|0.63648||
|2|2|* Used [public notebook](https://www.kaggle.com/code/batprem/home-credit-risk-mode-utility-scripts/notebook) features <br> * Designed LGBM XGB on feature subsets and batch predictions on multiple feature subsets <br> * Null cutoff = 70%|400|LGBM x 2 <br> XGB x 1|0.83245|0.63662||
|2|3|* Used [public notebook](https://www.kaggle.com/code/batprem/home-credit-risk-mode-utility-scripts/notebook) features <br> * Designed LGBM on feature subsets and batch predictions on multiple feature subsets <br> * Null cutoff = 80%|422|LGBM x 2|0.83483|0.64798||
|2|4|* Used [public notebook](https://www.kaggle.com/code/batprem/home-credit-risk-mode-utility-scripts/notebook) features <br> * Designed LGBM on feature subsets and batch predictions on multiple feature subsets <br> * Null cutoff = 95%|502|LGBM x 2|0.83615|0.65295|0.549|
|2|5|* Used [public notebook](https://www.kaggle.com/code/kononenko/metric-s-trick-home-credit-baseline-inference) features <br> * Designed LGBM XGB combination <br> * Null cutoff > 95% <br> * Used post-processed CV score also|502|LGBM x 4 <br> XGB x 1 <br> LAMA x 1|0.83510<br>0.83836|0.65261<br>0.66045|0.61|
|2|6|* Used [public notebook](https://www.kaggle.com/code/kononenko/metric-s-trick-home-credit-baseline-inference) features <br> * Null cutoff > 85%<br> * Used post-processed CV score also|440|LAMA x 1|0.82843|0.63397||



<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #b3e0ff; border-bottom: 8px solid #1a0d00" > PREPROCESSING<br> <div>

In [ ]:
%%time

class DataXformer:
    """
    This is a comprehensive preprocessing and data transformer class that does the below-
    1. consumes the input data tables
    2. creates secondary features
    3. ensues memory efficient outputs for the model
    """;

    def __init__(self, null_cutoff: float, cat_cutoff: int,
                 TrainTest: str      = "train",
                 sel_cols: list      = [],
                 cat_cols: list      = [],
                 **kwarg
                 ):

        self.TrainTest   = TrainTest;
        self.null_cutoff = null_cutoff;
        self.cat_cutoff  = cat_cutoff;
        self.target      = CFG.target;
        self.sel_cols    = sel_cols;
        self.cat_cols    = cat_cols;

        if self.TrainTest == "train":
            self.path = CFG.train_path;
        else:
            self.path = CFG.test_path;

        Utils.PrintColor(f"\n{'='*10} {self.TrainTest.upper()} MODE {'='*10}\n", color = Fore.RED);

    def _TypeCastCols(self, df: pl.DataFrame):
        """
        This method casts the columns into the desired dtypes with basic date handling too
        Input- df- pl.DataFrame:- input data table
        Output- df:- pl.DataFrame:- dataframe with type-casting
        """;

        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64));
            elif col in ["date_decision"] or col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date));
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64));
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String));

        return df;

    def _MakeDtFtre(self, df: pl.DataFrame):
        """
        This method creates date features from the provided dataframe
        Input- df- pl.DataFrame:- input data table
        Output- df- pl.DataFrame:- dataframe with date column FE
        """;

        for col in df.columns:
            if col.endswith("D"):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"));
                df = df.with_columns(pl.col(col).dt.total_days());
                df = df.with_columns([pl.col("date_decision").dt.month().alias("month_nb").cast(pl.Int8),
                                      pl.col("date_decision").dt.weekday().alias("weekday_nb").cast(pl.Int8),
                                     ]
                                    );
        return df.drop("date_decision", "MONTH");

    def _MakeAgg(self, df: pl.DataFrame):
        """
        This method makes a set of aggregate expressions for group by on case id for depth > 0 tables

        Note:-
        1. We make [max, min, first, last] aggregations for all columns
        2. We make mean aggregation for columns ending with [P, A, D]
        3. We make mode aggregations for columns ending with [M]

        Input - df- pl.DataFrame:- input data table
        Output- all_agg:- list of aggregate expressions to be used with group_by case_id
        """;

        all_agg = [];
        df_cols = df.columns;

        all_agg.extend([method(col).alias(f"{method.__name__}_{col}") \
                        for method in [pl.max, pl.min, pl.first, pl.last] \
                        for col in df_cols if col[-1] in ("P", "A", "D", "M", "T", "L") or "num_group" in col
                        ]
                       );
        all_agg.extend([pl.col(col).mean().alias(f"mean_{col}") for col in df_cols if col.endswith(("P", "A", "D"))]);
        all_agg.extend([pl.col(col).drop_nulls().mode().first().alias(f"mode_{col}") for col in df_cols if col.endswith("M")]);
        return df.group_by("case_id").agg(all_agg);

    def _PreProcessIpTbl(self, path:str, depth: int, isSingle: bool, **kwarg):
        """
        This method does the below-
        1. Creates chunks for file loads if we have multiple files (isSingle = False)
        2. Concatenates the chunks to a single file with typecasting
        3. Aggregating on case id for depth > 0 tables
        """;

        if isSingle == False:
            components = [];
            for path in glob(str(path)):
                components.append(pl.scan_parquet(path).pipe(self._TypeCastCols));
            df = pl.concat(components, how = "vertical_relaxed");
        else:
            df = pl.scan_parquet(path).pipe(self._TypeCastCols);

        if depth > 0:
            return df.pipe(self._MakeAgg);
        else:
            return df;

    @staticmethod
    def _ReduceMem(df: pd.DataFrame):
        "This method reduces memory for numeric columns in the dataframe";

        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', "uint16", "uint32", "uint64"];
        start_mem = df.memory_usage().sum() / 1024**2;

        for col in df.columns:
            col_type = df[col].dtypes

            if col_type in numerics:
                c_min = df[col].min();
                c_max = df[col].max();

                if "int" in str(col_type):
                    if c_min >= np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min >= np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min >= np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min >= np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min >= np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    if c_min >= np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

        end_mem = df.memory_usage().sum() / 1024**2;

        Utils.PrintColor(f"Start - end memory:- {start_mem:5.2f} - {end_mem:5.2f} Mb");
        return df;

    def _MakeModelData(self, df_base, depth_0, depth_1, depth_2, **kwarg):
        """
        This method aggregates the input tables and joins them to make a single model table for the next steps
        It converts the final table to a pandas dataframe for the next steps, reduces the memory consumption and selects relevant columns
        """;

        for i, df in enumerate(depth_0 + depth_1 + depth_2):
            df_base = df_base.join(df, how = "left", on = "case_id", suffix = f"_{i}");

        df = df_base.pipe(self._MakeDtFtre).collect().to_pandas();
        df = self._ReduceMem(df.replace([np.inf, -1*np.inf], np.NaN));

        if self.TrainTest.lower() == "train":
            Utils.PrintColor(f"---> Selecting training columns by nulls and category unique values",
                             color = Fore.CYAN
                             );

            drop_cols = [];
            null_cols = df.drop(columns = [self.target], errors = "ignore").isna().mean();
            drop_cols.extend(null_cols.loc[null_cols > self.null_cutoff].index.to_list());

            obj_cols = df.select_dtypes(include = "object").columns;
            for col in obj_cols:
                if df[col].nunique() > self.cat_cutoff or df[col].nunique() == 1:
                    drop_cols.append(col);
            cat_cols  = [c for c in obj_cols if c not in drop_cols];

            df = df.drop(columns = drop_cols, errors = "ignore");
            df[cat_cols] = df[cat_cols].astype("category");

        else:
            Utils.PrintColor(f"---> Selecting the test-set columns and aligning category columns with train data");
            df                = df[self.sel_cols];
            df[self.cat_cols] = df[self.cat_cols].astype("category");

        return df;

    def XformData(self, display_store: bool = False):
        """
        This is the cynosure method that aggregates all the inputs and prepares the FE dataset for the model training/ submission
        """;

        data_store = \
         {"df_base" : self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_base.parquet"), depth = 0, isSingle = True),

          "depth_0" : [self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_static_cb_0.parquet"), depth = 0, isSingle = True),
                       self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_static_0_*.parquet"), depth = 0, isSingle = False)
                      ],

          "depth_1": [self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_applprev_1_*.parquet"), depth = 1, isSingle = False),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_tax_registry_a_1.parquet"), depth = 1, isSingle = True),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_tax_registry_b_1.parquet"), depth = 1, isSingle = True),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_tax_registry_c_1.parquet"), depth = 1, isSingle = True),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_credit_bureau_b_1.parquet"), depth = 1, isSingle = True),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_other_1.parquet"), depth = 1, isSingle = True),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_person_1.parquet"), depth = 1, isSingle = True),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_deposit_1.parquet"), depth = 1, isSingle = True),
                      self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_debitcard_1.parquet"), depth = 1, isSingle = True)
                      ],

          "depth_2": [self._PreProcessIpTbl(os.path.join(self.path, f"{self.TrainTest}_credit_bureau_b_2.parquet"), depth = 2, isSingle = True)]
          };

        if display_store:
            Utils.PrintColor("\n---> Data store\n", color = Fore.CYAN);
            pprint(data_store, width = 200, depth = 3, indent = 5);

        df = self._MakeModelData(**data_store);
        Utils.PrintColor(f"\n---> {self.TrainTest.capitalize()} set details = {df.shape} | {df.memory_usage().sum()/ 10**6 :,.2f} Mb\n",
                         color = Fore.CYAN
                         );
        del data_store;

        if self.TrainTest.lower() == "train":
            cols = df.drop(columns = [CFG.target, "WEEK_NUM", "case_id"], errors = "ignore").columns;
            joblib.dump(cols, os.path.join(CFG.model_path, f"SelCols_E{CFG.exp_nb}V{CFG.version_nb}.pkl"));
            cat_cols = df.select_dtypes(include = "category").columns;
            joblib.dump(cat_cols, os.path.join(CFG.model_path, f"SelCatCols_E{CFG.exp_nb}V{CFG.version_nb}.pkl"));

            with np.printoptions(linewidth = 160):
                Utils.PrintColor("\n---> Train set columns\n");
                pprint(np.array(cols));
                Utils.PrintColor("\n---> Train set category columns\n", color = Fore.CYAN);
                pprint(np.array(cat_cols));
        else:
            pass;
        return df;

Utils.PrintColor(Utils.CleanMemory());



RAM usage = 0.8105 GB

CPU times: user 209 ms, sys: 0 ns, total: 209 ms

Wall time: 206 ms


In [ ]:
%%time

pp      = DataXformer(TrainTest = "train", null_cutoff = CFG.null_cutoff, cat_cutoff = CFG.cat_cutoff);
XYtrain = pp.XformData(display_store = False);
Utils.PrintColor(Utils.CleanMemory());

if CFG.test_req == "Y":
    XYtrain = XYtrain.groupby(["WEEK_NUM", CFG.target]).sample(frac = CFG.test_sample_frac).sort_index();
    XYtrain.index = range(len(XYtrain));
    Utils.PrintColor(f"---> Train shape after sampling = {XYtrain.shape}");

pp      = DataXformer(TrainTest   = "test",
                      null_cutoff = CFG.null_cutoff,
                      cat_cutoff  = CFG.cat_cutoff,
                      sel_cols    = XYtrain.drop(columns = [CFG.target]).columns.tolist(),
                      cat_cols    = XYtrain.select_dtypes(include = "category").columns.tolist(),
                      );
Xtest   = pp.XformData(display_store = False);

Utils.PrintColor(Utils.CleanMemory());



========== TRAIN MODE ==========



Start - end memory:- 10766.64 - 6943.36 Mb

---> Selecting training columns by nulls and category unique values



---> Train set details = (1526659, 440) | 2,099.21 Mb





---> Train set columns



array(['birthdate_574D', 'dateofbirth_337D', 'days120_123L', 'days180_256L', 'days30_165L', 'days360_512L', 'days90_310L', 'description_5085714M',

       'education_1103M', 'education_88M', 'firstquarter_103L', 'fourthquarter_440L', 'maritalst_385M', 'maritalst_893M', 'numberofqueries_373L',

       'pmtscount_423L', 'pmtssum_45A', 'requesttype_4525192L', 'responsedate_1012D', 'responsedate_4527233D', 'secondquarter_766L', 'thirdquarter_1082L',

       'actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L',

       'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P',

       'avgdbddpdlast

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #b3e0ff; border-bottom: 8px solid #1a0d00" > MODEL TRAINING<br> <div>

In [ ]:
%%time

model = TabularAutoML(task = Task('binary', loss = 'logloss', metric = 'auc'),
                      timeout   = 10000,
                      cpu_limit = 4,
                      gpu_ids   = '0',
                      general_params = {"use_algos": [["denselight"]]},
                      nn_params = {"n_epochs"       : 5,
                                   "bs"             : 128,
                                   "num_workers"    : 0,
                                   "path_to_save"   : None,
                                   "freeze_defaults": True,
                                   "cont_embedder"  : "cont",
                                   },
                      nn_pipeline_params = {"use_qnt": False, "use_te": False},
                      reader_params = {'n_jobs': 4,
                                       'cv'    : CFG.n_splits,
                                       'random_state': CFG.state,
                                       'advanced_roles': False
                                       },
                      );

oof_preds = \
model.fit_predict(XYtrain,
                  roles   = {'target': CFG.target,'group': "WEEK_NUM",'drop' : ['case_id', "WEEK_NUM"]},
                  verbose = 0
                  );



INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.


INFO:lightautoml.automl.presets.base:Task: binary



INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:

INFO:lightautoml.automl.presets.base:- time: 10000.00 seconds

INFO:lightautoml.automl.presets.base:- CPU: 4 cores

INFO:lightautoml.automl.presets.base:- memory: 16 GB



INFO:lightautoml.reader.base:Train data shape: (1526659, 440)



INFO:lightautoml.automl.base:Layer 1 train process start. Time left 9995.28 secs

DEBUG:lightautoml.ml_algo.dl_model:number of text features: 0 

DEBUG:lightautoml.ml_algo.dl_model:number of categorical features: 99 

DEBUG:lightautoml.ml_algo.dl_model:number of continuous features: 324 

INFO:lightautoml.ml_algo.base:Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 ...

DEBUG:lightautoml.ml_algo.base:Training params: {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': 

CPU times: user 4h 47min 3s, sys: 1min, total: 4h 48min 3s

Wall time: 1h 17min 23s


In [ ]:
%%time

Utils.PrintColor(f"OOF score = {Utils.ScoreMetric(XYtrain[CFG.target], oof_preds.data):.5f}");
stb_metric = Utils.StabilityMetric(XYtrain[["WEEK_NUM", CFG.target]].assign(score = oof_preds.data.flatten()));
Utils.PrintColor(f"Stability Metric = {stb_metric:.5f}\n\n", color = Fore.CYAN);

joblib.dump(model, os.path.join(CFG.model_path, f"E{CFG.exp_nb}V{CFG.version_nb}_DENSELIGHT.model"));

_ = Utils.CleanMemory();

OOF score = 0.82843

Stability Metric = 0.63397





CPU times: user 1.6 s, sys: 32 ms, total: 1.63 s

Wall time: 1.74 s
